# Plan and Execute

> - 참고 자료: https://wikidocs.net/270688
> - 관련 논문: https://arxiv.org/abs/2305.04091

Plan-and-Execute 에이전트는 먼저 달성할 목표를 구체적인 단계로 나누어 계획을 수립한 뒤, 그 계획을 순차적으로 실행하며 실행 과정에서 얻은 피드백을 반영해 동적으로 조정하는 구조를 갖추고 있습니다.

[ReAct](https://arxiv.org/abs/2210.03629) 에이전트는 사고와 행동을 매 단계 즉시 교차 실행하며 즉각적 피드백에 대응하는 반면, Plan-and-Execute 에이전트는 전체 목표를 세부 단계별로 계획 수립 후 순차 실행하며 피드백으로 계획을 동적으로 조정합니다.

In [1]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv("../.env", override=True)


def _set_env(var: str):
    env_value = os.environ.get(var)
    if not env_value:
        env_value = getpass.getpass(f"{var}: ")

    os.environ[var] = env_value


_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"
_set_env("OPENAI_API_KEY")

## 에이전트 정의

랭그래프에 사전 정의된 `create_react_agent`를 사용하여 ReAct 에이전트를 생성합니다.

In [11]:
from langgraph.prebuilt import create_react_agent
from langchain_tavily import TavilySearch

web_search_tool = TavilySearch(max_results=3)

agent_executor = create_react_agent(
    model="openai:gpt-4.1-mini",
    tools=[web_search_tool],
    prompt="당신은 언제나 친절하고 능숙하게 지원을 제공하며, 사용자의 목표 달성을 돕는 탁월한 조수입니다.",
)

In [12]:
agent_executor.invoke({"messages": [("user", "랭그래프에 대해서 웹검색 해주세요.")]})

{'messages': [HumanMessage(content='랭그래프에 대해서 웹검색 해주세요.', additional_kwargs={}, response_metadata={}, id='a8421bec-c953-4275-974e-ca91ce7f07c0'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CLy03HjiDdZh8Imec5JEB8rc', 'function': {'arguments': '{"query":"랭그래프","search_depth":"basic"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 1311, 'total_tokens': 1335, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_95d112f245', 'id': 'chatcmpl-CP6IB82y7FkwtTZrdjbnDAeL2nr4L', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--bd791a95-611a-4297-8aab-57ba10e74d97-0', tool_calls=[{'name': 'tavily_search', 'args': {'query':

## 상태 정의

In [13]:
from typing import TypedDict, Annotated
import operator


class State(TypedDict):
    input: Annotated[str, "사용자 요청"]
    plan: Annotated[list[str], "현재 계획"]
    past_steps: Annotated[list[tuple], operator.add]
    response: Annotated[str, "최종 응답"]

## 계획 노드

계획을 작성하는 노드를 작성합니다.

In [16]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI


class Plan(BaseModel):
    """계획 실행을 위한 단계별 분류"""

    steps: Annotated[
        list[str], Field(..., description="각 단계들은 정렬된 순서로 배치되어야 합니다")
    ]


def plan_node(state: State):
    system_prompt = (
        "목표를 달성하기 위해 간단한 단계별 계획을 세우세요."
        "각 단계 작업을 정확히 수행하면 정답이 도출되고 마지막 단계 결과가 최종 답안이 됩니다."
        "누락이나 중복 없이 필요한 정보만 포함하세요."
    )

    llm = ChatOpenAI(
        model="gpt-4.1-mini",
        temperature=0,
    ).with_structured_output(Plan)

    response = llm.invoke(
        [
            ("system", system_prompt),
            ("user", state["input"]),
        ]
    )
    return {"plan": response.steps}

In [17]:
plan_node(
    {"input": "LangGraph 의 핵심 장단점과 LangGraph 를 사용하는 이유는 무엇인가?"}
)

{'plan': ['LangGraph의 핵심 장점을 조사한다.',
  'LangGraph의 핵심 단점을 조사한다.',
  'LangGraph를 사용하는 이유를 정리한다.',
  '장단점과 사용하는 이유를 간결하게 요약한다.']}

## 재계획 노드

이전 결과를 바탕으로 계획을 다시 수립하는 노드를 작성합니다.

In [ ]:
from typing import Union


class Response(BaseModel):
    """사용자에 대한 응답"""

    response: str


class Act(BaseModel):
    """수행할 작업"""

    action: Annotated[
        Union[Response, Plan],
        Field(
            ...,
            description="수행할 작업입니다."
            "사용자에게 응답하려면, Response를 사용합니다."
            "답을 얻기 위해 추가 도구가 필요하다면, Plan을 사용합니다.",
        ),
    ]


def replan(state: State):
    system_prompt_template = """목표 달성을 위한 단순 단계별 계획을 수립하세요.
계획은 각 과업으로 구분되며, 과업 이행 시 정답 도출이 가능해야 합니다. 불필요 과업은 배제합니다.
최종 과업의 산출물이 곧 최종 답안입니다. 단계 누락을 금하고, 모든 필수 정보를 포함하세요.

## 현재 정보

### 목표: 
{input}

### 기존 계획: 
{plan}

#### 이행 경과:
{past_steps}

계획을 갱신하되, 미완료 과업만 보강하세요. 
잔여 과업이 없으면 즉시 결과를 제시하세요."""

    llm = ChatOpenAI(
        model="gpt-4.1-mini",
        temperature=0,
    ).with_structured_output(Act)

    response = llm.invoke(
        [
            (
                "system",
                system_prompt_template.format(
                    input=state["input"],
                    plan=state["plan"],
                    past_steps=state["past_steps"],
                ),
            )
        ]
    )

    if isinstance(response.action, Response):
        return {"response": response.action.response}
    else:
        next_plan = response.action.steps
        if len(next_plan) > 0:
            return {"plan": next_plan}
        else:
            return {"response": "더 이상 단계가 필요하지 않습니다."}

## 실행 노드

주어진 작업을 수행하고 결과를 반환합니다.

In [ ]:
def execute_note(state: State):
    prompt_template = (
        """다음 계획:{plan_str}\n\n당신은 [1단계. {task}]를 수행해야 합니다."""
    )

    plan = state["plan"]
    task = plan[0]

    response = agent_executor.invoke(
        {
            "messages": [
                (
                    "user",
                    prompt_template.format(
                        plan="\n".join(
                            f"{i + 1}. {step}" for i, step in enumerate(plan)
                        ),
                        task=task,
                    ),
                )
            ],
        }
    )

    return {
        "past_steps": [
            (
                task,
                response["messages"][-1].content,
            )
        ]
    }

## 리포트 생성

작업 내용을 바탕으로 리포트를 작성합니다.

In [ ]:
def generate_report_note(state: State):
    system_prompt_template = """당신에게는 목표와 이전에 수행된 단계들이 주어졌습니다. 
당신의 임무는 최종 보고서를 마크다운 형식으로 생성하는 것입니다.
최종 보고서는 전문적인 어조로 작성해야 합니다.

## 목표:

{input}

## 이전에 수행된 단계(질문과 답변 쌍):

{past_steps}

## 형식:

마크다운 형식으로 최종 보고서를 생성하세요."""

    past_steps = "\n\n".join(
        [
            f"Question: {past_step[0]}\n\nAnswer: {past_step[1]}\n\n####"
            for past_step in state["past_steps"]
        ]
    )

    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.7)

    response = llm.invoke(
        [
            (
                "system",
                system_prompt_template.format(
                    input=state["input"],
                    past_steps=past_steps,
                ),
            )
        ]
    )
